In [ ]:
# Install necessary packages
!pip install tensorflow matplotlib opencv-python

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Load your trained U-Net or DeepLabV3+ model
# model = tf.keras.models.load_model('path_to_your_model.h5')
from tensorflow.keras.models import Model
def create_gradcam_model(model, target_layer_name):
    target_layer = model.get_layer(target_layer_name).output
    return Model(inputs=model.input, outputs=[target_layer, model.output])

In [ ]:
def compute_gradcam(image_tensor, gradcam_model, class_index=1):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = gradcam_model(image_tensor)
        loss = tf.reduce_mean(predictions[..., class_index])
    grads = tape.gradient(loss, conv_outputs)
    weights = tf.reduce_mean(grads, axis=(1, 2))
    cam = tf.reduce_sum(tf.multiply(conv_outputs, tf.expand_dims(tf.expand_dims(weights, 1), 1)), axis=-1)
    cam = tf.nn.relu(cam)
    cam = tf.image.resize(cam[..., tf.newaxis], (image_tensor.shape[1], image_tensor.shape[2]))
    cam = cam / (tf.reduce_max(cam) + 1e-8)
    return cam.numpy()[0, ..., 0]

In [ ]:
def overlay_gradcam(image, cam):
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    overlay = heatmap * 0.4 + image
    return np.uint8(overlay)

In [ ]:
# Load an image (as [1, H, W, 3]) and run Grad-CAM
# image_tensor = tf.expand_dims(image, 0)
# gradcam_model = create_gradcam_model(model, 'target_layer_name')
# cam = compute_gradcam(image_tensor, gradcam_model)
# overlay = overlay_gradcam((image * 255).astype(np.uint8), cam)
# plt.imshow(overlay); plt.title('Grad-CAM Overlay'); plt.axis('off'); plt.show()